https://www.tensorflow.org/datasets/keras_example

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files = True,
    as_supervised = True,
    with_info = True
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Penggunaan Callback mencegah overfitting dan menghentikan training setelah akurasi terpenuhi
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('sparse_categorical_accuracy') >= 0.99):
      print("\nAkurasi di atas 99%, hentikan training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs = 20,
    validation_data=ds_test,
    callbacks = [myCallback()]
)

Epoch 1/20
469/469 [==============================] - 2s 4ms/step - loss: 0.3624 - sparse_categorical_accuracy: 0.9008 - val_loss: 0.2013 - val_sparse_categorical_accuracy: 0.9441
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1673 - sparse_categorical_accuracy: 0.9520 - val_loss: 0.1358 - val_sparse_categorical_accuracy: 0.9595
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1207 - sparse_categorical_accuracy: 0.9652 - val_loss: 0.1148 - val_sparse_categorical_accuracy: 0.9651
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0937 - sparse_categorical_accuracy: 0.9728 - val_loss: 0.0996 - val_sparse_categorical_accuracy: 0.9697
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0743 - sparse_categorical_accuracy: 0.9784 - val_loss: 0.0925 - val_sparse_categorical_accuracy: 0.9718
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0621 - sparse_categorica

In [ ]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_19 (Flatten)        (None, 784)               0         
                                                                 
 dense_38 (Dense)            (None, 128)               100480    
                                                                 
 dense_39 (Dense)            (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_pred_accuracy = model.predict(ds_test)

array([[ -7.3684936,  -5.2066674,  14.529613 , ...,  -8.7915535,
         -0.4172987,  -9.732165 ],
       [ 12.642114 , -10.44571  ,  -9.871433 , ...,  -6.725908 ,
         -7.2445345, -13.535431 ],
       [-11.015508 , -10.536512 ,  -5.1540227, ...,   1.191118 ,
         -4.427827 ,   4.580145 ],
       ...,
       [ -4.36959  ,  -9.670373 ,   2.6553156, ..., -13.101283 ,
         10.863406 ,  -4.7470045],
       [ 14.815834 ,  -8.850632 ,   6.381195 , ...,  -5.3990784,
        -11.11113  ,  -4.05085  ],
       [ -9.1409   ,  -6.964634 ,  -7.866238 , ..., -12.188226 ,
         -4.2098217,  -2.8811307]], dtype=float32)